In [1]:
"""
 Simple Multi-Layer Pecptron with Custom Data
 @Sungjoon Choi (sungjoon.choi@cpslab.snu.ac.kr)
"""
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
# Load them!
cwd = os.getcwd()
loadpath = cwd + "/data/trainingset.npz"
l = np.load(loadpath)

# See what's in here
l.files

# Parse data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]

print ("%d train images loaded" % (ntrain))
print ("%d test images loaded" % (ntest))
print ("%d dimensional input" % (dim))
print ("%d classes" % (nclass))

69 train images loaded
18 test images loaded
900 dimensional input
2 classes


In [3]:
# Parameters
learning_rate   = 0.001
training_epochs = 100
batch_size      = ntrain
display_step    = nclass

# Network Parameters
n_hidden_1 = 256 # 1st layer num features
n_hidden_2 = 128 # 2nd layer num features
n_input    = dim # data input (img shape: 28*28)
n_classes  = nclass # total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
    return (tf.sigmoid(tf.matmul(layer_2, _weights['out']) + _biases['out']))
    # return (tf.sigmoid(tf.matmul(layer_2, _weights['out']) + _biases['out']))
    
# Store layers weight & bias
stddev = 0.1 # <== This greatly affects accuracy!! 
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print ("Network Ready to Go!")

Network Ready to Go!


In [4]:
# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) # Softmax loss
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

# Initializing the variables
init = tf.initialize_all_variables()

In [5]:
# Launch the graph
sess = tf.Session()
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(ntrain/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        randidx = np.random.randint(ntrain, size=batch_size)
        batch_xs = trainimg[randidx, :]
        batch_ys = trainlabel[randidx, :]   
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})/total_batch
    # Display logs per epoch step
    if epoch % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys})
        print ("Training accuracy: %.3f" % (train_acc))

print ("Optimization Finished!")


Epoch: 000/100 cost: 0.690817058
Training accuracy: 0.522
Epoch: 002/100 cost: 0.685374737
Training accuracy: 0.536
Epoch: 004/100 cost: 0.675440788
Training accuracy: 0.594
Epoch: 006/100 cost: 0.685953081
Training accuracy: 0.551
Epoch: 008/100 cost: 0.678594172
Training accuracy: 0.594
Epoch: 010/100 cost: 0.675776839
Training accuracy: 0.609
Epoch: 012/100 cost: 0.668014824
Training accuracy: 0.623
Epoch: 014/100 cost: 0.675286949
Training accuracy: 0.565
Epoch: 016/100 cost: 0.654722035
Training accuracy: 0.696
Epoch: 018/100 cost: 0.656322002
Training accuracy: 0.667
Epoch: 020/100 cost: 0.617808342
Training accuracy: 0.783
Epoch: 022/100 cost: 0.635324121
Training accuracy: 0.783
Epoch: 024/100 cost: 0.619468510
Training accuracy: 0.855
Epoch: 026/100 cost: 0.593406022
Training accuracy: 0.812
Epoch: 028/100 cost: 0.620080769
Training accuracy: 0.681
Epoch: 030/100 cost: 0.577326953
Training accuracy: 0.754
Epoch: 032/100 cost: 0.599855006
Training accuracy: 0.783
Epoch: 034/100

In [6]:
test_acc = sess.run(accr, 
                    feed_dict={x: testimg, y: testlabel})
print ("Test accuracy: %.3f" % (test_acc))

Test accuracy: 0.833
